In [1]:
import torch
from chadhmm.hmm import MultinomialHMM, GaussianHMM, GaussianMixtureHMM, PoissonHMM
from chadhmm.utilities import constraints

In [2]:
hmm = MultinomialHMM(n_states=6,
                     n_features=4,
                     n_trials=1,
                     transitions=constraints.Transitions.ERGODIC)
train_seq = torch.randint(0,hmm.n_features,(1000,))
one_hot = hmm.n_trials * torch.nn.functional.one_hot(train_seq,4)

In [3]:
x_valid = hmm.to_observations(one_hot,lengths=[100,900])
hmm._compute_log_likelihood(x_valid)

log_alpha_vec = hmm._forward(x_valid)
concated_fwd = torch.stack([log_alpha[-1] for log_alpha in log_alpha_vec],1)

In [5]:
gamma, xi = hmm._compute_posteriors(x_valid)

In [15]:
gamma[0][:,0]

tensor([-0.9996, -2.9181, -8.5645, -8.8128, -7.4956, -1.7700, -2.2875, -2.7483,
        -7.6512, -3.5299, -2.5173, -3.6399, -7.9764, -3.3475, -2.4080, -3.6058,
        -7.9181, -3.0705, -3.2890, -2.6629, -2.5400, -2.3259, -1.8742, -1.6323,
        -1.9615, -3.3330, -3.0781, -7.6296, -3.0129, -2.6112, -8.0409, -3.0028,
        -2.9584, -8.2080, -3.2620, -8.5033, -8.1086, -3.2645, -2.3096, -2.8009,
        -1.9470, -1.5857, -1.8371, -2.6442, -2.7608, -7.6988, -3.1886, -3.4762,
        -2.9422, -7.8047, -2.4601, -2.5882, -2.0925, -3.0743, -2.3919, -2.8044,
        -2.6275, -1.8013, -2.1436, -2.2441, -3.5879, -8.6455, -7.9233, -1.9013,
        -1.6464, -2.1137, -2.2300, -3.0699, -2.4153, -2.6692, -2.3380, -2.3292,
        -3.4134, -3.1263, -8.3665, -8.2882, -2.4857, -1.9736, -1.9695, -3.5829,
        -7.9437, -3.0898, -3.2258, -2.4602, -2.0725, -2.5150, -7.2404, -2.1738,
        -3.2883, -2.9292, -2.9342, -2.9894, -8.4565, -8.2325, -3.7836, -8.0952,
        -2.4059, -3.4848, -8.0638, -2.39

In [17]:
xi[0].logsumexp(2)

tensor([[-0.9996, -1.4929, -4.2359, -1.5538, -1.9378, -3.2887],
        [-2.9181, -1.9305, -1.8507, -1.6641, -1.0336, -2.3159],
        [-8.5645, -1.4978, -1.3320, -1.1820, -2.3651, -2.1924],
        [-8.8128, -1.1996, -1.3401, -1.4884, -2.5569, -2.0141],
        [-7.4956, -1.2366, -1.1358, -1.7161, -2.5611, -2.0332],
        [-1.7700, -2.3342, -0.9171, -2.6492, -1.5069, -3.1993],
        [-2.2875, -3.3221, -0.6621, -2.9208, -1.3495, -3.4000],
        [-2.7483, -5.2131, -0.3337, -2.8334, -6.2650, -1.8738],
        [-7.6512, -2.3890, -0.6458, -2.5939, -2.1655, -1.6392],
        [-3.5299, -2.9172, -0.8409, -2.5796, -1.5628, -1.6099],
        [-2.5173, -5.0712, -0.5244, -2.8237, -6.5358, -1.3460],
        [-3.6399, -2.8482, -1.0655, -2.6855, -1.2900, -1.4794],
        [-7.9764, -2.0956, -0.8956, -1.6771, -2.3823, -1.6658],
        [-3.3475, -2.6393, -1.0009, -2.2799, -1.5496, -1.5547],
        [-2.4080, -4.8968, -0.5804, -2.6941, -6.4945, -1.2955],
        [-3.6058, -2.7918, -1.1239, -2.6

In [26]:
print(f'pi {hmm.pi.exp()}\n' +
      f'A  {hmm.A.exp()}\n' +
      f'B  {hmm.pdf.probs}')

pi tensor([0.2087, 0.0644, 0.3401, 0.2302, 0.1185, 0.0382], dtype=torch.float64)
A  tensor([[0.0525, 0.1070, 0.2997, 0.4123, 0.0408, 0.0878],
        [0.1350, 0.2310, 0.0935, 0.0475, 0.3413, 0.1518],
        [0.2604, 0.0510, 0.0108, 0.2147, 0.3672, 0.0959],
        [0.6609, 0.0364, 0.0029, 0.0844, 0.0982, 0.1171],
        [0.0218, 0.1631, 0.1115, 0.1767, 0.0340, 0.4928],
        [0.1135, 0.2171, 0.0736, 0.1355, 0.0936, 0.3666]], dtype=torch.float64)
B  tensor([[0.0045, 0.2309, 0.5774, 0.1871],
        [0.4840, 0.1541, 0.2927, 0.0692],
        [0.2219, 0.1030, 0.5484, 0.1268],
        [0.2135, 0.3896, 0.1298, 0.2672],
        [0.5318, 0.1766, 0.2723, 0.0192],
        [0.3236, 0.0676, 0.3649, 0.2439]], dtype=torch.float64)


In [27]:
hmm.fit(X=one_hot,max_iter=10,n_init=1,verbose=True)

Run 1 | Initialization | Score: -1428.37
Run 1 | Iteration: 1 | Score: -1385.12 | Delta: 43.25 | Converged = False
Run 1 | Iteration: 2 | Score: -1385.28 | Delta: -0.16 | Converged = False
Model converged after 3 iterations with log-likelihood: -1385.39


MultinomialHMM(
  (_params): ParameterDict(
      (A): Object of type: Categorical
      (emission_pdf): Object of type: Multinomial
      (pi): Object of type: Categorical
  )
)

In [ ]:
hmm.predict(one_hot)

In [ ]:
gmm = GaussianHMM(n_states=2,n_features=2)
train_seq = torch.rand((1000,2), dtype=torch.float64)
gmm.fit(X=train_seq,max_iter=5, n_init=1, verbose=True)

In [ ]:
phmm = PoissonHMM(n_states=2,n_features=2)
train_seq = torch.randint(1,100,(100,2))
phmm.fit(X=train_seq,max_iter=15,lengths=[50,50],n_init=1,verbose=False)

In [ ]:
gmmm = GaussianMixtureHMM(n_states=4,
                         n_components=3,
                         n_features=3)
train_seq = torch.rand((1000,3),dtype=torch.float64)
gmmm.fit(X=train_seq,max_iter=15, n_init=1, verbose=True)

In [ ]:
import torch
from chadhmm.hsmm import CategoricalHSMM, GaussianHSMM, GaussianMixtureHSMM, PoissonHSMM

In [ ]:
hsmm = CategoricalHSMM(n_states=5,
                       n_features=4,
                       max_duration=4)
train_seq = torch.randint(0, 4, (100,))
hsmm.fit(X=train_seq,max_iter=15, n_init=1)

In [ ]:
print(f'pi {hsmm.params.pi.exp().sum()}\n'+
      f'A  {hsmm.params.A.exp().sum(1)}\n' +
      f'D  {hsmm.params.D.exp().sum(1)}\n' +
      f'B  {hsmm.params.B.exp().sum(1)}')

In [ ]:
ghsmm = GaussianHSMM(n_states=5,
                  n_features=3,
                  max_duration=4)
train_seq = torch.rand((100,3), dtype=torch.float64)
ghsmm.fit(X=train_seq,max_iter=15, n_init=1, verbose=False)

In [ ]:
gmhsmm = GaussianMixtureHSMM(n_states=4,
                         n_components=3,
                         max_duration=4,
                         n_features=3)
train_seq = torch.rand((100,3),dtype=torch.float64)
gmhsmm.fit(X=train_seq,max_iter=15, n_init=1, verbose=False)

In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical, Multinomial

Multinomial(2,probs=torch.Tensor([0.2,0.8]))

In [ ]:
m = Multinomial(1,logits=hmm.pdf.logits)
m.log_prob(torch.Tensor([0,0,0,1])).exp()

In [ ]:
hmm.pdf.log_prob(train_seq[4])